In [1]:
import os.path
import sys
from bids.grabbids import BIDSLayout
import ciftify.utils
from ciftify.utils import run, get_number_cpus, section_header
import logging
from docopt import docopt
import pandas as pd
import ciftify.bidsapp.run as run_script

In [2]:
def freesurfer_is_complete(settings, participant_label):
    '''returns bool of wether or not the wmparc.mgz was found'''
    fs_sub_dir = os.path.join(settings.fs_dir,
                                 'sub-{}'.format(participant_label))
    finished_fs = os.path.isfile(os.path.join(fs_sub_dir, 'mri', 'wmparc.mgz'))
    return finished_fs

def ciftify_log_endswith_done(ciftify_log):
    '''return true with the ciftify log file exists and ends with the word Done'''
    if not os.path.isfile(ciftify_log):
        return False
    with open(ciftify_log, 'r') as f:
        lines = f.read().splitlines()
        last_line = lines[-3]
        is_done = True if 'Done' in last_line else False
    return is_done

def has_ciftify_recon_all_run(settings, participant_label):
    '''determine if ciftify_recon_all has already completed'''
    ciftify_log = os.path.join(settings.ciftify_work_dir,
                        'sub-{}'.format(participant_label),
                        'cifti_recon_all.log')
    return ciftify_log_endswith_done(ciftify_log)

def has_ciftify_fmri_run(subject, fmriname, settings):
    '''determine if ciftify_recon_all has already completed'''
    ciftify_log = os.path.join(settings.ciftify_work_dir,
                        'sub-{}'.format(subject),
                        'MNINonLinear', 'Results', fmriname,
                        'ciftify_subject_fmri.log')
    return ciftify_log_endswith_done(ciftify_log)

In [3]:
def check_fmriprep_html_for_errors(subject, settings):
    fmriprep_report = os.path.join(settings.fmriprep_dir, 'sub-{}.html'.format(subject))
    try: 
        with open(fmriprep_report, 'r') as report_contents:
            if 'No errors to report' in report_contents.read():
                return True
    except:
        return False
    return False

def build_dataframe_of_completed_runs(settings):
    '''checks all runs for completeness and builds a dataframe'''
    df = settings.bids_layout.as_data_frame()

    for subject in df.subject.unique():
        df.loc[df.loc[:, 'subject']== subject, 'freesurfer_done'] = freesurfer_is_complete(settings, subject)
        df.loc[df.loc[:, 'subject']== subject, 'ciftify_anat_done'] = has_ciftify_recon_all_run(settings, subject)

    for idx in df.index:
        if df.loc[idx, 'type'] == "bold":
            subject = df.loc[idx, 'subject']
            if 'session' in df.columns:
                bold_input_list = settings.bids_layout.get(subject = subject,
                                                           type="bold",
                                                           session = df.loc[idx, 'session'],
                                                           task = df.loc[idx, 'task'],
                                                           extensions=["nii.gz", "nii"])
            else: 
                bold_input_list = settings.bids_layout.get(subject = subject,
                                                           type="bold",
                                                           task = df.loc[idx, 'task'],
                                                           extensions=["nii.gz", "nii"])
            try:
                bold_input = bold_input_list[0]
                bold_preproc, fmriname = run_script.find_bold_preproc(bold_input, settings)
                df.loc[idx, 'fmriprep_done'] = os.path.isfile(bold_preproc)
                df.loc[idx, 'fmriprep_no_errors'] = check_fmriprep_html_for_errors(subject, settings)
                df.loc[idx, 'ciftify_fmri_done'] = has_ciftify_fmri_run(subject, fmriname, settings)
            except:
                print('row {} is weird'.format(idx))
    return df

In [4]:
def print_fmriprep_deleting_code(df, settings):
    '''note: this is only good if for ad hoc deleting when removing the aroma error code..'''
    subs_to_delete_fmriprep = df.loc[df.loc[:, 'fmriprep_no_errors']==False, 'subject']
    for sub in subs_to_delete_fmriprep.unique():
        print('rm -r {}/sub-{}'.format(settings.fmriprep_dir, sub))
        print('rm -r {}/sub-{}.html'.format(settings.fmriprep_dir, sub))


In [5]:
def print_ciftify_anat_deleting_code(df, settings):
    subs_to_delete_ciftify_anat = df.loc[df.loc[:, 'ciftify_anat_done']==False, 'subject']
    for sub in subs_to_delete_ciftify_anat.unique():
        print('rm -r {}/sub-{}'.format(settings.ciftify_work_dir, sub))

In [6]:
def print_ciftify_fmri_deleting_code(df, settings):
    '''prints code to remove half done ciftify_subject_fmri runs'''
    df_step1 = df.loc[df.loc[:, 'ciftify_anat_done']==True, :]
    df_step2 = df_step1.loc[df_step1.loc[:, 'ciftify_fmri_done']==False, :]

    for idx in df_step2.index:
        if 'session' in df.columns:
            bold_input_list = settings.bids_layout.get(subject = df_step2.loc[idx, 'subject'],
                                                       type="bold",
                                                       session = df_step2.loc[idx, 'session'],
                                                       task = df_step2.loc[idx, 'task'],
                                                       extensions=["nii.gz", "nii"])
        else:
            bold_input_list = settings.bids_layout.get(subject = df_step2.loc[idx, 'subject'],
                                                       type="bold",
                                                       task = df_step2.loc[idx, 'task'],
                                                       extensions=["nii.gz", "nii"])

        bold_input = bold_input_list[0]
        bold_preproc, fmriname = run_script.find_bold_preproc(bold_input, settings)
        print('rm -r {}/sub-{}/MNINonLinear/Results/{}'.format(settings.ciftify_work_dir, bold_input.subject, fmriname))

In [7]:
arguments_zhh  = docopt(run_script.__doc__, ['/scratch/dmiranda/BIDS_ZHH', '/scratch/edickie/saba_PINT/ciftify_fmriprep/ZHH/out/', 'participant', '--task=rest'])

In [8]:
settings_zhh = run_script.Settings(arguments_zhh)

cmd: bids-validator /scratch/dmiranda/BIDS_ZHH 
 Failed with returncode 127
/bin/sh: 1: bids-validator: not found



In [9]:
df_zhh = build_dataframe_of_completed_runs(settings_zhh)
df_rest_zhh = df_zhh.loc[df_zhh.loc[:, 'task']=="rest", :]

row 92 is weird


In [10]:
df_rest_zhh

,path,modality,session,subject,task,type,freesurfer_done,ciftify_anat_done,fmriprep_done,fmriprep_no_errors,ciftify_fmri_done
0,/scratch/dmiranda/BIDS_ZHH/sub-10944/ses-01/fu...,func,01,10944,rest,bold,True,True,True,True,True
2,/scratch/dmiranda/BIDS_ZHH/sub-9104/ses-01/fun...,func,01,9104,rest,bold,True,True,True,True,True
4,/scratch/dmiranda/BIDS_ZHH/sub-7735/ses-01/fun...,func,01,7735,rest,bold,True,True,True,True,True
6,/scratch/dmiranda/BIDS_ZHH/sub-8523/ses-01/fun...,func,01,8523,rest,bold,True,True,True,True,True
8,/scratch/dmiranda/BIDS_ZHH/sub-3123/ses-01/fun...,func,01,3123,rest,bold,True,True,True,True,True
10,/scratch/dmiranda/BIDS_ZHH/sub-5692/ses-02/fun...,func,02,5692,rest,bold,True,True,True,True,True
12,/scratch/dmiranda/BIDS_ZHH/sub-9878/ses-01/fun...,func,01,9878,rest,bold,True,True,True,True,True
14,/scratch/dmiranda/BIDS_ZHH/sub-9799/ses-01/fun...,func,01,9799,rest,bold,True,True,True,True,True
16,/scratch/dmiranda/BIDS_ZHH/sub-9061/ses-01/fun...,func,01,9061,rest,bold,True,True,True,True,True
18,/scratch/dmiranda/BIDS_ZHH/sub-11323/ses-02/fu...,func,02,11323,rest,bold,True,True,True,True,True


In [11]:
import datetime

today = datetime.datetime.today().strftime("%Y-%m-%d")


In [12]:
print_fmriprep_deleting_code(df_rest_zhh, settings_zhh)

rm -r /scratch/edickie/saba_PINT/ciftify_fmriprep/ZHH/out/fmriprep/sub-11242
rm -r /scratch/edickie/saba_PINT/ciftify_fmriprep/ZHH/out/fmriprep/sub-11242.html
rm -r /scratch/edickie/saba_PINT/ciftify_fmriprep/ZHH/out/fmriprep/sub-6729
rm -r /scratch/edickie/saba_PINT/ciftify_fmriprep/ZHH/out/fmriprep/sub-6729.html


In [13]:
df_rest_zhh.to_csv(os.path.join(settings_zhh.ciftify_work_dir, 'ciftify_runs_complete_{}.csv'.format(today)))

In [14]:
#print_ciftify_fmri_deleting_code(df_rest_zhh, settings_zhh)
print_ciftify_anat_deleting_code(df_rest_zhh, settings_zhh)

In [15]:
print_ciftify_fmri_deleting_code(df_rest_zhh, settings_zhh)

rm -r /scratch/edickie/saba_PINT/ciftify_fmriprep/ZHH/out/ciftify/sub-11242/MNINonLinear/Results/ses-02_task-rest_bold
rm -r /scratch/edickie/saba_PINT/ciftify_fmriprep/ZHH/out/ciftify/sub-6729/MNINonLinear/Results/ses-01_task-rest_bold


In [ ]:
subs_to_delete_fmriprep = df_rest_zhh.loc[df_rest_zhh.loc[:, 'fmriprep_no_errors']==False, 'subject']
for sub in subs_to_delete_fmriprep.unique():
    print(sub)

In [ ]:
subs_to_delete_ciftify_anat = df_rest.loc[df_rest.loc[:, 'ciftify_anat_done']==False, 'subject']

In [ ]:
arguments_cobre  = docopt(run_script.__doc__, ['/scratch/edickie/saba_PINT/ciftify_fmriprep/COBRE/COBRE', '/scratch/edickie/saba_PINT/ciftify_fmriprep/COBRE/out/', 'participant', '--task=rest'])

In [ ]:
settings_cobre = run_script.Settings(arguments_cobre)
df = build_dataframe_of_completed_runs(settings_cobre)
df_rest = df.loc[df.loc[:, 'task']=="rest", :]

In [ ]:
df_rest

In [ ]:
df_rest.to_csv(os.path.join(settings_cobre.ciftify_work_dir, 'ciftify_runs_complete_{}.csv'.format(today)))

In [ ]:
print_ciftify_fmri_deleting_code(df_rest, settings_cobre)

In [ ]:
arguments_cnp  = docopt(run_script.__doc__, ['/scratch/edickie/saba_PINT/bids_in/ds000030/ds000030_R1.0.5/', '/scratch/edickie/saba_PINT/ciftify_fmriprep/ds000030_R1.0.5/out/', 'participant', '--task=rest'])

In [ ]:
settings_cnp = run_script.Settings(arguments_cnp)
df_cnp = build_dataframe_of_completed_runs(settings_cnp)
df_rest_cnp = df_cnp.loc[df_cnp.loc[:, 'task']=="rest", :]

In [ ]:
df_rest_cnp.to_csv(os.path.join(settings_cnp.ciftify_work_dir, 'ciftify_runs_complete_{}.csv'.format(today)))